# GoogleNet

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Base directory containing the image data

# Define the data generators
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Set the image directory and other parameters for the generators
train_dir = '/content/drive/MyDrive/BACH-dataset/train_dir'
test_dir = '/content/drive/MyDrive/BACH-dataset/test_dir'
val_dir = '/content/drive/MyDrive/BACH-dataset/val_dir'
target_size = (224, 224)
batch_size = 8

# Create the train, test, and validation generators
train_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

test_generator = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

val_generator = val_datagen.flow_from_directory(
    directory=val_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
)

Found 318 images belonging to 4 classes.
Found 38 images belonging to 4 classes.
Found 44 images belonging to 4 classes.


In [8]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD

base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add your own classification layers on top with dropout
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Add a dropout layer with 50% dropout rate
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)  # Add another dropout layer with 50% dropout rate
predictions = Dense(4, activation='softmax')(x)  # Replace 'num_classes' with the number of your classes

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Define the SGD optimizer with standard hyperparameters
learning_rate = 0.01
momentum = 0.9
learning_rate_schedule = SGD(learning_rate=learning_rate, momentum=momentum, nesterov=True)

# Compile the model with the SGD optimizer
model.compile(optimizer=learning_rate_schedule, loss='categorical_crossentropy', metrics=['accuracy'])


# Train the model on your dataset
# Make sure to provide your own dataset and preprocessing steps
history = model.fit(train_generator, epochs=50, validation_data=val_generator)

Epoch 1/50
40/40 [==============================] - 80s 2s/step - loss: 2.9019 - accuracy: 0.3553 - val_loss: 3.3384 - val_accuracy: 0.2727
Epoch 2/50
40/40 [==============================] - 71s 2s/step - loss: 1.7081 - accuracy: 0.4811 - val_loss: 1.2119 - val_accuracy: 0.4545
Epoch 3/50
40/40 [==============================] - 70s 2s/step - loss: 1.2798 - accuracy: 0.5000 - val_loss: 1.1189 - val_accuracy: 0.5227
Epoch 4/50
40/40 [==============================] - 70s 2s/step - loss: 1.1829 - accuracy: 0.5031 - val_loss: 1.1368 - val_accuracy: 0.4773
Epoch 5/50
40/40 [==============================] - 75s 2s/step - loss: 1.0520 - accuracy: 0.5283 - val_loss: 1.0217 - val_accuracy: 0.5682
Epoch 6/50
40/40 [==============================] - 72s 2s/step - loss: 0.9540 - accuracy: 0.5943 - val_loss: 0.9781 - val_accuracy: 0.5909
Epoch 7/50
40/40 [==============================] - 82s 2s/step - loss: 1.0540 - accuracy: 0.5943 - val_loss: 1.0812 - val_accuracy: 0.4773
Epoch 8/50
40/40 [==